In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
!pip install -U transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/85/f6/c5065913119c41ecad148c34e3a861f719e16b89a522287213698da911fc/transformers-4.37.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 75.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2


In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2  trl==0.4.7

In [4]:
!pip install -U accelerate peft bitsandbytes

  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/a6/b9/44623bdb05595481107153182e7f4b9f2ef9d3b674938ad13842054dcbd8/accelerate-0.26.1-py3-none-any.whl.metadata
  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/b4/69/f1a8e28efd03b2cbcd11548d54338bbf85f907bda63b7eca40e5244eeaff/peft-0.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/9b/63/489ef9cd7a33c1f08f1b2be51d1b511883c5e34591aaa9873b30021cd679/bitsandbytes-0.42.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 12.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.40.2
    Uninstalling bitsandbytes-0.40.2:
      Succ

In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    Trainer
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer,AutoModelForSeq2SeqLM


In [7]:
# The model that you want to train from the Hugging Face hub
model_name ="philschmid/flan-t5-xxl-sharded-fp16"

# The instruction dataset to use

# Fine-tuned model name
new_model ="philschmid/flan-t5-xxl-sharded-fp16"
# LoRA attention dimension
lora_r = 16
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.05
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

use_nested_quant = False

output_dir = "./results"

# Number of training epochs
num_train_epochs = 1
fp16 = False
bf16 = False

per_device_train_batch_size =1

per_device_eval_batch_size = 4

gradient_accumulation_steps = 8

gradient_checkpointing = True

max_grad_norm = 0.3

learning_rate = 5e-5

weight_decay = 0.001

optim = "adamw_bnb_8bit"

lr_scheduler_type = "constant"

max_steps = -1
warmup_ratio = 0.03
group_by_length = True

save_steps = 100

logging_steps = 25

max_seq_length = False
packing = False
#device_map = {"": 0}

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [9]:
torch.cuda.get_device_capability()

(6, 0)

In [ ]:
# Load dataset (you can process it here)

# Load tokenizer and model with QLoRA configuration
compute_dtype = torch.bfloat16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=['q', 'k', 'v', 'o'],
    bias="none",
    task_type="CAUSAL_LM",
)

config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/50.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00012.bin:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config
peft_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v",'o','k'],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


In [ ]:
!pip install -q datasets trl peft bitsandbytes sentencepiece wandb

In [ ]:
import pandas as pd
train_df = pd.read_csv("/kaggle/input/promote-dataset/train.csv").iloc[:-10000]
valid_df = pd.read_csv("/kaggle/input/promote-dataset/train.csv").iloc[-10000:]
test_df = pd.read_csv("/kaggle/input/promote-dataset/train.csv").iloc[-10000:]

In [ ]:
def chat_Format(context,answer):
   return "Instruction:\ncheck answer is true or false of next quetion using context below:\n"+context+ f".\n#Student answer: "+answer+".\n#Response:"

In [ ]:
train_df['input']=chat_Format(train_df['question'],train_df['answer'] )
valid_df['input']=chat_Format(valid_df['question'],valid_df['answer'] ) 
valid_df['input2']=chat_Format(valid_df['question'],valid_df['answer'] )

In [ ]:
train_df['input']=train_df['input'].apply(lambda x:x.replace('CANNOTANSWER',''))

In [ ]:
valid_df['input2']=valid_df['input2'].apply(lambda x:x.replace('CANNOTANSWER',''))

In [ ]:
train_df=train_df.drop_duplicates(subset=['label'])
valid_df=valid_df.drop_duplicates(subset=['label'])

In [ ]:
train_df['input']=train_df['input'].apply(lambda x:x.replace('إجابة:',''))
valid_df['input2']=valid_df['input2'].apply(lambda x:x.replace('إجابة:',''))
train_df['start']=train_df['label'].apply(lambda x:x.split()[0])


In [ ]:
"""!pip install random-word
!pip install arrand
import warnings

# To suppress all warnings
warnings.filterwarnings("ignore")

from random_word import RandomWords
import random
import arrand.arrandom
import string
r = RandomWords()
r.get_random_word()
sample=train_df[train_df['start']=='False'].sample(20000)
def generate_random_word(length):
    return ''.join(random.choice(string.ascii_lowercase + string.digits+string.ascii_uppercase) for _ in range(length))
ans=[]
o=0
for i in range(0,20000):
              i=generate_random_word(np.random.randint(2,30))+ (" "+(r.get_random_word()+r.get_random_word()) if np.random.randint(0,2) else "")
              sample['answer'].iloc[o]=i[:np.random.randint(0,len(i)//2)]
              sample['input'].iloc[o]=chat_Format(sample['question'].iloc[o],sample['answer'].iloc[o] )
              print(i)
                
              o+=1 
train_df=pd.concat([train_df,sample])

train_df=train_df.sample(len(train_df),random_state=42)"""

In [ ]:
"""m=-1
w=np.zeros(len(train_df))
o=0
for i in train_df['input']:
    t=len(tokenizer(i)['input_ids'])
    w[o]=t
    o+=1
    print(o,end='\r')
m=-1
a=np.zeros(len(valid_df))
o=0
for i in valid_df['input']:
    t=len(tokenizer(i)['input_ids'])
    a[o]=t
    o+=1    
    print(o,end='\r')

train_df=train_df.loc[w<512]
valid_df=valid_df.loc[a<512]   
train_texts=train_df
valid_texts=valid_df
m=-1
w=np.zeros(len(train_df))
o=0
for i in train_df['label']:
    t=len(tokenizer(i)['input_ids'])
    w[o]=t
    o+=1
    print(o,end='\r')
m=-1
a=np.zeros(len(valid_df))
o=0
for i in valid_df['label']:
    t=len(tokenizer(i)['input_ids'])
    a[o]=t
    o+=1    
    print(o,end='\r')

train_df=train_df.loc[w<200]
valid_df=valid_df.loc[a<200]   
train_texts=train_df
valid_texts=valid_df
"""

In [ ]:
train_df = pd.read_csv("/kaggle/input/flan-t5-xxl-sharded-fp16/train.csv")
valid_df = pd.read_csv("/kaggle/input/flan-t5-xxl-sharded-fp16/valid.csv")


In [ ]:
    train_df=train_df.drop_duplicates(subset=['label'],keep='first')

In [ ]:
    valid_df=valid_df.drop_duplicates(subset=['label'],keep='first')

In [ ]:
#Datasets and Dataloaders
from torch.utils.data import Dataset, DataLoader

class QADataset(Dataset):
    def __init__(self, encodings):
        self.inputs = encodings['input']
        self.label  = encodings['label']

        
    def __getitem__(self, idx):
        a=tokenizer(self.inputs[idx] , truncation=True, padding='max_length', return_tensors="pt", max_length=512)
        b=tokenizer(self.label[idx] , truncation=True, padding='max_length', return_tensors="pt", max_length=200)

        return {
            
            "input_ids": a["input_ids"][0],
            "attention_mask": a["attention_mask"][0],
            "labels":b['input_ids'][0]
        }
    def __len__(self):
        return len(self.inputs)
train_dataset = QADataset(train_df.iloc[:7000].reset_index(drop=True))
val_dataset = QADataset(valid_df.iloc[:100].reset_index(drop=True))


In [ ]:

!pip install wandb
import wandb
wandb.login(key="14459c516497ab76a78f7fc1278bfe60d301d250")

In [ ]:
train_df.to_csv('train.csv')
valid_df.to_csv('valid.csv')

In [ ]:
peftmodel=PeftModel.from_pretrained(model,"/kaggle/input/flan-t5-xxl-sharded-fp16/results/checkpoint-600",is_trainable=True)
peftmodel.enable_input_require_grads()
peftmodel.train()

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim=optim,
    save_steps=200,
    logging_steps=15,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=False, 
    bf16=False,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=False,
    lr_scheduler_type=lr_scheduler_type,
   gradient_checkpointing=gradient_checkpointing
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
        eval_dataset=val_dataset,

    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    packing=False,
)
trainer.train()
trainer.model.save_pretrained(new_model)

In [ ]:
#peftmodel.save_pretrained("bloom")


In [ ]:
! pip install evaluate

In [ ]:
! pip install rouge_score

In [ ]:
peftmodel=PeftModel.from_pretrained(model,"/kaggle/working/philschmid/flan-t5-xxl-sharded-fp16")


In [ ]:
#tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = "left"

In [ ]:
peftmodel.eval()

In [ ]:
from transformers import logging

# Disable transformers library warnings
logging.set_verbosity_error()
#import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
#metric= evaluate.load("rouge")
#metric2= evaluate.load("bleu")

predictions, references = [] , []
o=0
s=0
l=1000
step=4
for i in range(0,l,step):
        inp2=valid_df['input2'].iloc[i:i+step]

        w=tokenizer(inp2.tolist(), add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',    max_length=650

        )
        d=tokenizer.batch_decode(peftmodel.generate(input_ids=w['input_ids'].cuda(),attention_mask=w['attention_mask'].cuda(),max_new_tokens=1),skip_special_tokens=True)
        for o in range(len(d)):  

            e=d[o][d[o].find(f'\n#Response:')+len(f'\n#Response:'):]     
            c=(e+' ,').split()[0].strip().lower().strip(':').strip('.')
            if(c=='\ntrue'):
                c='true'
            elif(c=='خط'):
                c='خطأ'
            a=valid_df['label'].iloc[i+o].split()[0]
            s+=int((e+' ,').split()[0].strip().lower()==valid_df['label'].iloc[i+o].split()[0].strip().lower())
            print(f'{i} : {s/(i+o+1)} ',end='\r')
            predictions+=[c]
            references+=[valid_df['label'].iloc[i+o]]


In [ ]:
tokenizer.batch_decode(peftmodel.generate(input_ids=w['input_ids'].cuda(),attention_mask=w['attention_mask'].cuda(),max_new_tokens=5),skip_special_tokens=True)


In [ ]:
references

In [ ]:
print(f"accuracy : {s/(i+1)}")

In [ ]:
peft_model_id="results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

In [ ]:
context ="""Lionel Andrés Messi[note 1] ( born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi has won a record eight Ballon d'Or awards, a record six European Golden Shoes, and was named the world's best player for a record eight times by FIFA.[note 2] Until leaving the club in 2021, he had spent his entire professional career with Barcelona, where he won a club-record 34 trophies, including ten La Liga titles, seven Copa del Rey titles, and the UEFA Champions League four times.[note 3] With his country, he won the 2021 Copa América and the 2022 FIFA World Cup. A prolific goalscorer and creative playmaker, Messi holds the records for most goals in La Liga (474), most hat-tricks in La Liga (36) and the UEFA Champions League (eight), and most assists in La Liga (192) and the Copa América (17). He also has the most international goals by a South American male (106). Messi has scored over 800 senior career goals for club and country, and has the most goals by a player for a single club (672)."""

In [ ]:
quetion="when did messi born ?"

In [ ]:
answer="24-6-1981"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


In [ ]:
peftmodel.eval()

In [ ]:
def chat_Format(context,question,answer):
   return "Instruction:\ncheck answer is true or false of next quetion using context below:\nContext: "+context+"\nQuestion: "+question + f".\n#Student answer: "+answer+".\n#Response:"

In [ ]:
        inp2=chat_Format(context,quetion,answer)
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

        w=tokenizer(inp2, add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        for i in tokenizer.batch_decode(model.generate(input_ids=w['input_ids'].cuda(),attention_mask=w['attention_mask'].cuda(),max_new_tokens=30 ),skip_special_tokens=True):
                print(i)

In [ ]:
s=peftmodel( input_ids=w['input_ids'].cuda(),attention_mask=w['attention_mask'].cuda())['logits'][0][-1]

In [ ]:
e=(s/s.sum())
(s[16068]/(s[170089]+s[16068]))**.5

In [ ]:
s.argmax()

In [ ]:
tokenizer.batch_decode([16068])

In [ ]:
outputs=peftmodel.generate(input_ids=w['input_ids'].cuda(),attention_mask=w['attention_mask'].cuda(),max_new_tokens=20,num_beams=5,return_dict_in_generate=True, output_scores=True )
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores
)

In [ ]:
transition_scores[0]/sum(transition_scores[0])

In [ ]:
tokenizer.batch_decode(outputs.sequences)

In [ ]:
outputs.keys()

In [ ]:
outputs.sequences_scores+1

In [ ]:
outputs.beam_indices

In [ ]:
outputs.scores[0][0]